# Not Used!!

- Exploring options for address normalization.
- But abandoned because of poor results


In [ ]:
import json
import os
from pathlib import Path

import pandas as pd
import seaborn as sns

# Settings
sns.set_theme(style="whitegrid")
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 100)

json_dir = "../data/license_json"
source_dir = Path("../data/license_text")
target_dir = Path("../data/license_text_fix")

## 1. Load Data

Read all `.json` files from the `license_json` directory and combine them into a single DataFrame.

In [ ]:
data = []

if os.path.exists(json_dir):
    files = [f for f in os.listdir(json_dir) if f.endswith(".json")]
    print(f"Found {len(files)} JSON files.")

    for filename in files:
        with open(os.path.join(json_dir, filename)) as f:
            try:
                record = json.load(f)
                data.append(record)
            except Exception as e:
                print(f"Error loading {filename}: {e}")
else:
    print(f"Error: {json_dir} directory not found.")

df = pd.DataFrame(data)
print(f"DataFrame created with {len(df)} rows and {len(df.columns)} columns.")

In [ ]:

import requests


def call_nominatim(call_type, params):
    """
    Fetches a address related to this code
    """
    NOMINATIM_URL = "http://127.0.0.1:8080"

    url = f"{NOMINATIM_URL}/{call_type}"

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors (4xx or 5xx)
        route_data = response.json()
        return route_data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching route: {e}")
        return None


def geocode(query):
    try:
        params = {"q": query, "format": "json", "limit": 1}

        response = call_nominatim("search", params)

        # Expecting a list of dicts
        if isinstance(response, list) and len(response) > 0:
            first = response[0]

            lat = first.get("lat")
            lon = first.get("lon")

            # Ensure both values exist
            if lat is not None and lon is not None:
                return (float(lat), float(lon))
    except Exception as e:
        print(f"⚠️ Geocoding failed for '{query}': {e}")

    return (None, None)


def reverse_geocode(lat, lon):
    response = (None, None, None)
    try:
        params = {"lat": lat, "lon": lon, "format": "json", "addressdetails": 1}

        response = call_nominatim("reverse", params)

        # Expecting a list of dicts
        if isinstance(response, dict):
            addr_str = None
            addresstype = response.get("addresstype", None)
            address = response.get("address", None)
            if address:
                house_number = address.get("house_number", "")
                road = address.get("road", "")
                town = address.get("town", "")
                state = address.get("state", "")
                postcode = address.get("postcode", "")
                amenity = address.get("amenity", "")
                addr_str = f"{house_number} {road}, {town}"
            return (addr_str, addresstype, amenity)

    except Exception as e:
        print(f"⚠️ Reverse Geocoding failed for '[{lat},{lon}]': {e}")

    return response

In [ ]:
unique_licenses = df["license_number"].unique()
unique_licenses

In [ ]:
df["minutes_date"] = pd.to_datetime(df["minutes_date"], errors="coerce")

df_sorted = df.sort_values(by="minutes_date", ascending=False)
df_unique_newest = df_sorted.drop_duplicates("license_number", keep="first")

In [ ]:
# Make a copy
df_geo = df_unique_newest.copy()
cols_to_drop = [
    "minutes_date",
    "business_name",
    "dba_name",
    "alcohol_type",
    "manager",
    "attorney",
    "status",
    "status_detail",
    "hours",
    "details",
    "entity_number",
    "file_name",
]
df_geo = df_geo.drop(columns=cols_to_drop)
df_geo
# Limit to first 5 rows
# df_geo = df_geo.head(5)

In [ ]:
import pandas as pd

# Add empty columns
df_geo[["lat", "lon", "house_number", "road", "town", "state", "postcode"]] = None

for idx, row in df_geo.iterrows():
    address = row["address"]

    # --- Forward geocode ---
    lat, lon = geocode(address)

    df_geo.at[idx, "lat"] = lat
    df_geo.at[idx, "lon"] = lon

    # --- Reverse geocode if we got valid coords ---
    if lat is not None and lon is not None:
        addr_str, addresstype, amenity = reverse_geocode(lat, lon)

        # Your reverse_geocode returns a string and some metadata,
        # but we want structured fields, so call again properly:
        response = call_nominatim(
            "reverse", {"lat": lat, "lon": lon, "format": "json", "addressdetails": 1}
        )

        if isinstance(response, dict):
            address_details = response.get("address", {})

            df_geo.at[idx, "house_number"] = address_details.get("house_number")
            df_geo.at[idx, "road"] = address_details.get("road")
            df_geo.at[idx, "town"] = (
                address_details.get("town")
                or address_details.get("city")
                or address_details.get("village")
            )
            df_geo.at[idx, "state"] = address_details.get("state")
            df_geo.at[idx, "postcode"] = address_details.get("postcode")

In [ ]:
df_geo

In [ ]:
import csv

# Ensure columns are strings and saved with quotes
df_geo["zipcode"] = df_geo["zipcode"].astype(str)
df_geo["postcode"] = df_geo["postcode"].astype(str)

df_geo.to_csv("address_test.csv", index=False, quoting=csv.QUOTE_NONNUMERIC)